In [1]:
from random import shuffle

from IPython.display import display
from ipywidgets.widgets import Valid, VBox, Layout

from primes import least_prime_factor, make_sample

# Magnitude of primes that take a few seconds to check
#
# machine  magnitude
# RPI4     2 ** 49
# X250     2 ** 53
# YOGA9    2 ** 57
# M2MAX    2 ** 57
# VIVO     2 ** 63

MAGNITUDE = 2**57

sample = make_sample(MAGNITUDE)
shuffle(sample)

def make_cell(n, lpf):
        return Valid(
            value = lpf == n,
            description = f'{n:_}',
            readout = f'{lpf:_}' if lpf else '\N{Hourglass with Flowing Sand}',
            layout = Layout(width='90%'),
            style = {'description_width' : '200px'},
        )

In [2]:
%%time
cells = [make_cell(n, 0) for n in sample]

table = VBox(cells) 
display(table)

def update_table():
    for i, n in enumerate(sample):
        lpf = least_prime_factor(n)
        cells[i] = make_cell(n, lpf)
        table.children = cells

update_table()

CPU times: user 33.1 s, sys: 19 ms, total: 33.2 s
Wall time: 33.3 s


In [3]:
from concurrent import futures

from primes import lpf_pair
from table import Table
        

In [4]:
%%time

table = Table(sample)
table.display()

with futures.ProcessPoolExecutor() as pool:
    tasks = [pool.submit(lpf_pair, n) for n in sample]
    for future in futures.as_completed(tasks):
        n, lpf = future.result()
        table.update(n, lpf)

CPU times: user 48 ms, sys: 35.1 ms, total: 83.1 ms
Wall time: 6.12 s
